# CHESS OUTCOME PREDICTION — PRE-PROCESSING

**Goal:** prepare a clean, sequence-only dataset for deep learning.

**Cut-off:** first 20 full moves (40 plies).

**Input:** `data/chess_games_subset.csv` (uses `AN` for moves, `Result` for labels)  
**Output:** `data/chess_games_clean.csv`, `data/chess_games_clean_meta.json`

**Process:** load subset → tokenise `AN` → strip move numbers/result tokens → keep first 40 plies → add `plies_processed`, `cutoff_reached`, `captures_in_first_20_moves`, `checks_in_first_20_moves` → leakage checks → save.

In [20]:
# ==============================================
# 1. Imports & Paths
# ==============================================
from pathlib import Path
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# ----------------------------------------------
# 1.1 Ensure python-chess
# ----------------------------------------------
try:
    import chess
except ModuleNotFoundError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-chess", "--quiet"])
    import chess
print("python-chess OK.")

# ----------------------------------------------
# 1.2 Config (sequence only)
# ----------------------------------------------
CUTOFF_FULL_MOVES = 20
CUTOFF_PLIES = CUTOFF_FULL_MOVES * 2
MOVE_COL = f"moves_first{CUTOFF_FULL_MOVES}_san"
CAPTURES_COL = f"captures_in_first_{CUTOFF_FULL_MOVES}_moves"
CHECKS_COL = f"checks_in_first_{CUTOFF_FULL_MOVES}_moves"
SEQUENCE_ONLY = True

# ----------------------------------------------
# 1.3 Paths
# ----------------------------------------------
NB_DIR   = Path.cwd()
DATA_DIR = (NB_DIR / "../data").resolve()
OUT_DIR  = DATA_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)

RAW_SUBSET = DATA_DIR / "chess_games_subset.csv"
CLEAN_CSV  = DATA_DIR / "chess_games_clean.csv"
META_JSON  = DATA_DIR / "chess_games_clean_meta.json"

print("Data dir:", DATA_DIR)
print("Input :", RAW_SUBSET)
print("Output:", CLEAN_CSV)
print(f"Cutoff: {CUTOFF_FULL_MOVES} moves ({CUTOFF_PLIES} plies)")

python-chess OK.
Data dir: E:\Github Projects\chess-outcome-prediction\data
Input : E:\Github Projects\chess-outcome-prediction\data\chess_games_subset.csv
Output: E:\Github Projects\chess-outcome-prediction\data\chess_games_clean.csv
Cutoff: 20 moves (40 plies)


In [21]:
# ==============================================
# 2. Load Subset
# ==============================================
df_raw = pd.read_csv(RAW_SUBSET)

# ----------------------------------------------
# 2.1 Detect Columns
# ----------------------------------------------
cols = {c.lower(): c for c in df_raw.columns}

an_col = None
for k in ["an", "moves", "algebraic", "algebraicnotation"]:
    if k in cols:
        an_col = cols[k]
        break
if an_col is None:
    raise ValueError("Moves column (AN) not found. Expected a column like 'AN' or 'Moves'.")

label_col = None
for k in ["result", "target", "winner", "outcome"]:
    if k in cols:
        label_col = cols[k]
        break
if label_col is None:
    raise ValueError("Label column not found. Expected 'Result' or similar.")

# ----------------------------------------------
# 2.2 Clean Labels
# ----------------------------------------------
def map_label(v):
    s = str(v).strip().lower()
    if s in {"1-0", "white", "w"}:
        return "white"
    if s in {"0-1", "black", "b"}:
        return "black"
    if s in {"1/2-1/2", "draw", "d"}:
        return "draw"
    return np.nan

df = pd.DataFrame(
    {
        "moves_raw": df_raw[an_col].astype(str),
        "target": df_raw[label_col].map(map_label),
    }
).dropna(subset=["moves_raw", "target"]).reset_index(drop=True)

print(df.shape)
df.head(3)

(142303, 2)


,moves_raw,target
0,1. f4 d5 2. g3 c5 3. Bg2 Nc6 4. Nf3 Bf5 5. O-O...,white
1,1. e4 b6 2. d4 Bb7 3. Bd3 e6 4. f4 d6 5. Nf3 h...,white
2,1. d4 Nf6 2. c4 e6 3. Nc3 c5 4. d5 d6 5. e3 e5...,black


In [22]:
# ==============================================
# 3. Extract First 20 Moves (40 plies)
# ==============================================

# ----------------------------------------------
# 3.1 Helpers
# ----------------------------------------------
RESULT_TOKENS = {"1-0", "0-1", "1/2-1/2"}

def tokenize_an(s: str):
    toks = str(s).replace("\r", " ").replace("\n", " ").split()
    keep = []
    for t in toks:
        if t.endswith(".") and t[:-1].isdigit():
            continue
        if t == "...":
            continue
        if t in RESULT_TOKENS:
            continue
        keep.append(t)
    return keep

def count_captures(seq): 
    return sum("x" in m for m in seq)

def count_checks(seq): 
    return sum(("+" in m) or ("#" in m) for m in seq)

# ----------------------------------------------
# 3.2 Build Sequence Columns
# ----------------------------------------------
tokens = df["moves_raw"].apply(tokenize_an)
tokens_cut = tokens.apply(lambda t: t[:CUTOFF_PLIES])

df[MOVE_COL] = tokens_cut.apply(lambda t: " ".join(t))
df["plies_processed"] = tokens_cut.apply(len)
df["cutoff_reached"] = (df["plies_processed"] == CUTOFF_PLIES).astype(int)
df[CAPTURES_COL] = tokens_cut.apply(count_captures)
df[CHECKS_COL] = tokens_cut.apply(count_checks)

df = df.dropna(subset=[MOVE_COL, "target"]).reset_index(drop=True)

print(df.shape)
df[[MOVE_COL, "plies_processed", "target"]].head(3)

(142303, 7)


,moves_first20_san,plies_processed,target
0,f4 d5 g3 c5 Bg2 Nc6 Nf3 Bf5 O-O e6 d3 g6 Be3 h...,33,white
1,e4 b6 d4 Bb7 Bd3 e6 f4 d6 Nf3 h6 O-O a6 Qe2 Ne...,28,white
2,d4 Nf6 c4 e6 Nc3 c5 d5 d6 e3 e5 Nf3 Be7 Bd3 Nb...,40,black


In [23]:
# ==============================================
# 4. Assemble Working Frame (sequence-only, no leakage)
# ==============================================

# ----------------------------------------------
# 4.1 Select Columns
# ----------------------------------------------
keep_cols = [MOVE_COL, "plies_processed", "cutoff_reached", CAPTURES_COL, CHECKS_COL, "target"]
missing = [c for c in keep_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

X = df[keep_cols].copy()

# ----------------------------------------------
# 4.2 Leakage Guard
# ----------------------------------------------
banned = {
    "Result","result","winner","Winner","termination","Termination",
    "num_moves","NumMoves","Opening","opening","ECO","eco","Moves","AN",
    "UTCDate","UTCTime","Event","TimeControl","White","Black",
    "WhiteElo","BlackElo","WhiteRatingDiff","BlackRatingDiff"
}
present_banned = [c for c in X.columns if c in banned]
assert not present_banned, f"Leaky columns present in X: {present_banned}"

# ----------------------------------------------
# 4.3 Types
# ----------------------------------------------
X[MOVE_COL] = X[MOVE_COL].fillna("").astype(str)
X["target"] = X["target"].astype(str)
for c in ["plies_processed", "cutoff_reached", CAPTURES_COL, CHECKS_COL]:
    X[c] = pd.to_numeric(X[c], errors="coerce")

print("Working frame:", X.shape)
display(X.head(3))

Working frame: (142303, 6)


,moves_first20_san,plies_processed,cutoff_reached,captures_in_first_20_moves,checks_in_first_20_moves,target
0,f4 d5 g3 c5 Bg2 Nc6 Nf3 Bf5 O-O e6 d3 g6 Be3 h...,33,0,9,0,white
1,e4 b6 d4 Bb7 Bd3 e6 f4 d6 Nf3 h6 O-O a6 Qe2 Ne...,28,0,5,2,white
2,d4 Nf6 c4 e6 Nc3 c5 d5 d6 e3 e5 Nf3 Be7 Bd3 Nb...,40,1,7,0,black


In [24]:
# ==============================================
# 5. Impute & Downcast (sequence-only)
# ==============================================
import numpy as np

def mem_mb(df):
    return df.memory_usage(deep=True).sum() / (1024**2)

print(f"Memory before: {mem_mb(X):.2f} MB")

# ----------------------------------------------
# 5.1 Impute Numerics
# ----------------------------------------------
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
X.replace([np.inf, -np.inf], np.nan, inplace=True)
for c in num_cols:
    med = X[c].median()
    X[c] = X[c].fillna(med)

# ----------------------------------------------
# 5.2 Downcast Numerics
# ----------------------------------------------
for c in num_cols:
    if pd.api.types.is_integer_dtype(X[c]):
        X[c] = pd.to_numeric(X[c], downcast="integer")
    elif pd.api.types.is_float_dtype(X[c]):
        X[c] = pd.to_numeric(X[c], downcast="float")

# ----------------------------------------------
# 5.3 Preserve Sequence & Target Types
# ----------------------------------------------
X[MOVE_COL] = X[MOVE_COL].fillna("").astype(str)
X["target"] = X["target"].astype(str)

print(f"Memory after: {mem_mb(X):.2f} MB")
display(X.dtypes.to_frame("dtype").T)
display(X.head(3))

Memory before: 40.61 MB
Memory after: 37.35 MB


,moves_first20_san,plies_processed,cutoff_reached,captures_in_first_20_moves,checks_in_first_20_moves,target
dtype,object,int8,int8,int8,int8,object


,moves_first20_san,plies_processed,cutoff_reached,captures_in_first_20_moves,checks_in_first_20_moves,target
0,f4 d5 g3 c5 Bg2 Nc6 Nf3 Bf5 O-O e6 d3 g6 Be3 h...,33,0,9,0,white
1,e4 b6 d4 Bb7 Bd3 e6 f4 d6 Nf3 h6 O-O a6 Qe2 Ne...,28,0,5,2,white
2,d4 Nf6 c4 e6 Nc3 c5 d5 d6 e3 e5 Nf3 Be7 Bd3 Nb...,40,1,7,0,black


In [25]:
# ==============================================
# 6. Sanity Checks (leakage / balance / NAs / sequence)
# ==============================================

# ----------------------------------------------
# 6.1 Leakage Guard
# ----------------------------------------------
banned = {
    "Result","result","winner","Winner","termination","Termination",
    "num_moves","NumMoves","Opening","opening","ECO","eco","Moves","AN",
    "UTCDate","UTCTime","Event","TimeControl","White","Black",
    "WhiteElo","BlackElo","WhiteRatingDiff","BlackRatingDiff"
}
present_banned = [c for c in X.columns if c in banned]
assert not present_banned, f"Leaky columns present: {present_banned}"
assert MOVE_COL in X.columns

# ----------------------------------------------
# 6.2 Class Balance
# ----------------------------------------------
print("Target distribution (counts):")
display(X["target"].value_counts())
print("Target distribution (percent):")
display((X["target"].value_counts(normalize=True) * 100).round(2))

# ----------------------------------------------
# 6.3 NA Audit
# ----------------------------------------------
na_counts = X.isna().sum()
na_counts = na_counts[na_counts > 0]
if not na_counts.empty:
    print("Columns with missing values after imputation:")
    display(na_counts)
else:
    print("No missing values remaining")

# ----------------------------------------------
# 6.4 Sequence Checks
# ----------------------------------------------
total = len(X)
empty_seq = (X[MOVE_COL].str.len() == 0).sum()
at_cutoff = (X["plies_processed"] == CUTOFF_PLIES).sum()
over_cutoff = (X["plies_processed"] > CUTOFF_PLIES).sum()

print(f"Empty sequences: {empty_seq}/{total}")
print(f"At cutoff ({CUTOFF_PLIES} plies): {at_cutoff}/{total}")
print(f"Over cutoff (> {CUTOFF_PLIES}): {over_cutoff}")

display(X[[MOVE_COL, "plies_processed", "target"]].head(3))

Target distribution (counts):


target
white    70988
black    65801
draw      5514
Name: count, dtype: int64

Target distribution (percent):


target
white    49.89
black    46.24
draw      3.87
Name: proportion, dtype: float64

No missing values remaining
Empty sequences: 0/142303
At cutoff (40 plies): 119353/142303
Over cutoff (> 40): 0


,moves_first20_san,plies_processed,target
0,f4 d5 g3 c5 Bg2 Nc6 Nf3 Bf5 O-O e6 d3 g6 Be3 h...,33,white
1,e4 b6 d4 Bb7 Bd3 e6 f4 d6 Nf3 h6 O-O a6 Qe2 Ne...,28,white
2,d4 Nf6 c4 e6 Nc3 c5 d5 d6 e3 e5 Nf3 Be7 Bd3 Nb...,40,black


In [26]:
# ==============================================
# 7. Save Cleaned Data & Metadata
# ==============================================

# ----------------------------------------------
# 7.1 Write CSV
# ----------------------------------------------
X.to_csv(CLEAN_CSV, index=False)
print("Saved CSV:", CLEAN_CSV)

# ----------------------------------------------
# 7.2 Write Metadata
# ----------------------------------------------
meta = {
    "cutoff_full_moves": int(CUTOFF_FULL_MOVES),
    "cutoff_plies": int(CUTOFF_PLIES),
    "rows": int(len(X)),
    "columns": list(X.columns),
    "move_col": MOVE_COL,
    "captures_col": CAPTURES_COL,
    "checks_col": CHECKS_COL,
    "class_distribution": X["target"].value_counts().to_dict(),
}
import json
with open(META_JSON, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)
print("Saved meta:", META_JSON)

# ----------------------------------------------
# 7.3 Quick Preview
# ----------------------------------------------
print(X.shape)
display(X.head(3))

Saved CSV: E:\Github Projects\chess-outcome-prediction\data\chess_games_clean.csv
Saved meta: E:\Github Projects\chess-outcome-prediction\data\chess_games_clean_meta.json
(142303, 6)


,moves_first20_san,plies_processed,cutoff_reached,captures_in_first_20_moves,checks_in_first_20_moves,target
0,f4 d5 g3 c5 Bg2 Nc6 Nf3 Bf5 O-O e6 d3 g6 Be3 h...,33,0,9,0,white
1,e4 b6 d4 Bb7 Bd3 e6 f4 d6 Nf3 h6 O-O a6 Qe2 Ne...,28,0,5,2,white
2,d4 Nf6 c4 e6 Nc3 c5 d5 d6 e3 e5 Nf3 Be7 Bd3 Nb...,40,1,7,0,black
